***1. Обучить любую модель классификации на датасете IRIS до применения PCA (2 компоненты) и после него. Сравнить качество классификации по отложенной выборке.***

In [121]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier

In [122]:
def standard_scale(x):
    res = (x - x.mean(axis=0)) / x.std(axis=0)
    return res

In [123]:
iris = datasets.load_iris()
X = iris.data
target = iris.target
X = X.astype(float)
X = standard_scale(X)

In [124]:
# Разобьем выборку на обучающую и тестовую

train_data, test_data, train_labels, test_labels = train_test_split(X, 
                                                                    target, 
                                                                    test_size=0.3,
                                                                    random_state=1)

In [125]:
# Создаём модель леса из сотни деревьев (100 по умолчанию)
model = RandomForestClassifier(max_depth = 4,
                               max_features = None)
# Обучаем на тренировочных данных
model.fit(train_data, train_labels)


train_predictions = model.predict(train_data)
test_predictions = model.predict(test_data)

In [126]:
from sklearn.metrics import accuracy_score

# Рассчитываем accuracy

print(f"accuracy на тренировочных данных: {accuracy_score(train_predictions, train_labels)}\naccuracy на тестовых данных:{accuracy_score(test_predictions, test_labels)}")


accuracy на тренировочных данных: 1.0
accuracy на тестовых данных:0.9555555555555556


Теперь примененяем PCA (2 компоненты) 

Для реализации метода главных компонент нужно :
- найти собственные значения матрицы $X^{T}X$;
- отобрать $d$ максимальных;
- составить матрицу $W^{T}$, столбцы которой будут являться собственными векторами, соответствующими отобранным собственным значениям, расположенным в порядке убывания;
- получить новую матрицу "объекты-признаки", умножив исходную матрицу $X$ на матрицу весов $W$ :

$$Z=XW.$$

In [127]:
train_data.shape

(105, 4)

In [128]:
import itertools

indices = [0, 1, 2, 3]
comb_of_indices = list(itertools.combinations(indices, 2)) 
for i in comb_of_indices:
    print(f"Корреляция между признаком {i[0]} и {i[1]}: {np.corrcoef(train_data[i[0]], train_data[i[1]])[0][1]}")
for i in indices:
    print(f"Корреляция между признаком {i} и целевой переменной: {np.corrcoef(train_data[:, i], train_labels)[0][1]}")

Корреляция между признаком 0 и 1: -0.8437233930683575
Корреляция между признаком 0 и 2: -0.9458041967327759
Корреляция между признаком 0 и 3: -0.9796474724420249
Корреляция между признаком 1 и 2: 0.9715341143373748
Корреляция между признаком 1 и 3: 0.9306523801434927
Корреляция между признаком 2 и 3: 0.9876697877825587
Корреляция между признаком 0 и целевой переменной: 0.7841502769135629
Корреляция между признаком 1 и целевой переменной: -0.35725469728601256
Корреляция между признаком 2 и целевой переменной: 0.9507340057925742
Корреляция между признаком 3 и целевой переменной: 0.9572681043789399


In [129]:
X_red = X.copy()

In [130]:
eig_val, eig_vecs = np.linalg.eig(X_red .T @ X_red)
display(eig_val, eig_vecs)

array([437.77467248, 137.10457072,  22.01353134,   3.10722546])

array([[ 0.52106591, -0.37741762, -0.71956635,  0.26128628],
       [-0.26934744, -0.92329566,  0.24438178, -0.12350962],
       [ 0.5804131 , -0.02449161,  0.14212637, -0.80144925],
       [ 0.56485654, -0.06694199,  0.63427274,  0.52359713]])

In [131]:
vec = eig_vecs[:, :2]
vec

array([[ 0.52106591, -0.37741762],
       [-0.26934744, -0.92329566],
       [ 0.5804131 , -0.02449161],
       [ 0.56485654, -0.06694199]])

In [132]:
# Оцениваем объем потерянной информации

eig_sum = sum(eig_val)
[(i / eig_sum) * 100 for i in sorted(eig_val, reverse=True)]

[72.96244541329992, 22.85076178670176, 3.668921889282871, 0.5178709107154739]

Таким образом, наши большие векторы описывают 72.96 и 22.85%, а меньшие - 3.66 и 0.51%. Отбросив меньшие векторы и спроецировав данные на большие, мы потеряем меньше 5% информации.

In [133]:
Z = np.dot(X_red, vec)
Z[:15, :]

array([[-2.26470281, -0.4800266 ],
       [-2.08096115,  0.67413356],
       [-2.36422905,  0.34190802],
       [-2.29938422,  0.59739451],
       [-2.38984217, -0.64683538],
       [-2.07563095, -1.48917752],
       [-2.44402884, -0.0476442 ],
       [-2.23284716, -0.22314807],
       [-2.33464048,  1.11532768],
       [-2.18432817,  0.46901356],
       [-2.1663101 , -1.04369065],
       [-2.32613087, -0.13307834],
       [-2.2184509 ,  0.72867617],
       [-2.6331007 ,  0.96150673],
       [-2.1987406 , -1.86005711]])

In [134]:
Z.T @ Z

array([[4.37774672e+02, 2.77347589e-14],
       [2.77347589e-14, 1.37104571e+02]])

In [135]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_red)
X_reduced[:15, :]                            

array([[-2.26470281,  0.4800266 ],
       [-2.08096115, -0.67413356],
       [-2.36422905, -0.34190802],
       [-2.29938422, -0.59739451],
       [-2.38984217,  0.64683538],
       [-2.07563095,  1.48917752],
       [-2.44402884,  0.0476442 ],
       [-2.23284716,  0.22314807],
       [-2.33464048, -1.11532768],
       [-2.18432817, -0.46901356],
       [-2.1663101 ,  1.04369065],
       [-2.32613087,  0.13307834],
       [-2.2184509 , -0.72867617],
       [-2.6331007 , -0.96150673],
       [-2.1987406 ,  1.86005711]])

In [142]:
# Разобьем выборку на обучающую и тестовую

train_data_2, test_data_2, train_labels_2, test_labels_2 = train_test_split(Z, 
                                                                    target, 
                                                                    test_size=0.3,
                                                                   random_state=1)


In [143]:
# Создаём модель леса из сотни деревьев (100 по умолчанию)
model_2 = RandomForestClassifier(max_depth = 4,
                               max_features = None)
# Обучаем на новых тренировочных данных
model_2.fit(train_data_2, train_labels_2)


train_predictions_2 = model_2.predict(train_data_2)
test_predictions_2 = model_2.predict(test_data_2)

In [144]:
# Рассчитываем accuracy

print(f"accuracy на тренировочных данных: {accuracy_score(train_predictions_2, train_labels_2)}\naccuracy на тестовых данных:{accuracy_score(test_predictions_2, test_labels_2)}")


accuracy на тренировочных данных: 0.9809523809523809
accuracy на тестовых данных:0.9333333333333333


Результат обучения до PCA:
     accuracy на тренировочных данных: 1.0,
     accuracy на тестовых данных:0.9555555555555556

Результат обучения после PCA:
     accuracy на тренировочных данных: 0.9809523809523809,
     accuracy на тестовых данных:0.9555555555555556
     
Качество классификации на тренировочных данных оказалось примерно одинаковым, на тестовых данных - абсолютно одинаковым.

****2.* Написать свою реализацию метода главных компонент с помощью сингулярного разложения с использованием функции numpy.linalg.svd()***

Для реализации понижения размерности методом главных компонент с помощью SVD нужно:
- найти сингулярное разложение $X^TX$;
- сформировать из столбцов матрицы $V$, соответствующих $d$ наибольшим сингулярным числам, матрицу весов $W$;
- получить новую матрицу "объекты-признаки", умножив исходную матрицу $X$ на матрицу весов $W$:

$$Z=XW.$$

In [139]:
u, d, v_t = np.linalg.svd(X_red .T @ X_red)
v = v_t.T
print(d)
print(v)

[437.77467248 137.10457072  22.01353134   3.10722546]
[[-0.52106591 -0.37741762  0.71956635  0.26128628]
 [ 0.26934744 -0.92329566 -0.24438178 -0.12350962]
 [-0.5804131  -0.02449161 -0.14212637 -0.80144925]
 [-0.56485654 -0.06694199 -0.63427274  0.52359713]]


In [140]:
W = v[:, :2]
W

array([[-0.52106591, -0.37741762],
       [ 0.26934744, -0.92329566],
       [-0.5804131 , -0.02449161],
       [-0.56485654, -0.06694199]])

In [141]:
Z_2 = np.dot(X_red, W)
Z_2[:15, :]

array([[ 2.26470281, -0.4800266 ],
       [ 2.08096115,  0.67413356],
       [ 2.36422905,  0.34190802],
       [ 2.29938422,  0.59739451],
       [ 2.38984217, -0.64683538],
       [ 2.07563095, -1.48917752],
       [ 2.44402884, -0.0476442 ],
       [ 2.23284716, -0.22314807],
       [ 2.33464048,  1.11532768],
       [ 2.18432817,  0.46901356],
       [ 2.1663101 , -1.04369065],
       [ 2.32613087, -0.13307834],
       [ 2.2184509 ,  0.72867617],
       [ 2.6331007 ,  0.96150673],
       [ 2.1987406 , -1.86005711]])

In [145]:
# Разобьем выборку на обучающую и тестовую

train_data_3, test_data_3, train_labels_3, test_labels_3 = train_test_split(Z_2, 
                                                                    target, 
                                                                    test_size=0.3,
                                                                   random_state=1)

In [146]:
# Создаём модель леса из сотни деревьев (100 по умолчанию)
model_3 = RandomForestClassifier(max_depth = 4,
                               max_features = None)
# Обучаем на новых тренировочных данных
model_3.fit(train_data_3, train_labels_3)


train_predictions_3 = model_3.predict(train_data_3)
test_predictions_3 = model_3.predict(test_data_3)

In [147]:
# Рассчитываем accuracy

print(f"accuracy на тренировочных данных: {accuracy_score(train_predictions_3, train_labels_3)}\naccuracy на тестовых данных:{accuracy_score(test_predictions_3, test_labels_3)}")


accuracy на тренировочных данных: 0.9904761904761905
accuracy на тестовых данных:0.9555555555555556
